# Imports & Settings


In [2]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from deep_translator import GoogleTranslator
import re 
from math import isnan
import utilities
import helpers

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

In [3]:
# dtype={'type': str} prevents being confused with data type for large data sets
train = pd.read_csv('data/train.csv', index_col='id', dtype={'type': str})
test = pd.read_csv('data/test.csv', index_col='id', dtype={'type': str})
train_translated = pd.read_csv('data/train_translated.csv', dtype={'type': str})
test_translated = pd.read_csv('data/test_translated.csv', index_col='id', dtype={'type': str})
combined_data = pd.read_csv('data/combined_data.csv', index_col='id', dtype={'type': str})
combined_data_translated = pd.read_csv('data/combined_data_translated.csv', index_col='id', dtype={'type': str})
combined_data_fully_translated = pd.read_csv('data/combined_data_fully_translated.csv', index_col='id', dtype={'type': str})
prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})
test_prep = pd.read_csv('data/test_prepared.csv', index_col='id', dtype={'type': str})
train_prep = pd.read_csv('data/train_prepared.csv', index_col='id', dtype={'type': str})
low = pd.read_csv('data/prep_low_thres.csv', index_col='id', dtype={'type': str})
high = pd.read_csv('data/prep_high_thres.csv', index_col='id', dtype={'type': str})

# Delete unneeded features


In [9]:
data = prep.copy()

In [13]:
data.drop(columns=['full_nr', 'name','commentary','country_and_unit','text','parameter','unit','value','legend','initial_info','additional_text'], inplace=True)

In [15]:
data.to_csv('data/prep.csv')

In [11]:
data = prep.copy()

In [12]:
data.drop(columns=['film frame IN mm²', 'photo format IN mm²', 'negative format IN mm²', 'text_features'], inplace=True)

In [13]:
data.to_csv('data/prep.csv')

# rename for xgboost

In [6]:
data = high.copy()

In [7]:
data.rename(columns={'location_määramata [Narva-Jõesuu]':'location_määramata (Narva-Jõesuu)', 'location_määramata [Tallinn]':'location_määramata (Tallinn)', 'location_määramata [Tartu]':'location_määramata (Tartu)', 'location_määramata [Raplamaa]':'location_määramata (Raplamaa)'}, inplace=True)

In [5]:
data.to_csv('data/prep_high_thres.csv')

In [11]:
data = low.copy()

In [12]:
for i in data.columns:
    if '>' in i:
        data.rename(columns={i:i.replace('>','')}, inplace=True)
for i in data.columns:
    if '<' in i:
        data.rename(columns={i:i.replace('<','')}, inplace=True)

In [13]:
data.to_csv('data/prep_low_thres.csv')

# Nan ->0

In [7]:
data = prep.copy()

#for col in data.columns:
 #   if col.hasnan:
  #      data[col] = data[col].replace(np.nan, 0)

data = data.replace(np.nan, 0)
print (data.isnull().sum().sum())
print (data)

0
              ks  musealia_seria_nr  musealia_queue_nr  collection_queue_nr  \
id                                                                            
232170     118.0            12150.0              115.0                 44.0   
2251378    102.0            12584.0               19.0                  1.0   
2070466      0.0                0.0                0.0                 52.0   
4085096  41691.0                0.0                0.0                  0.0   
2697904    238.0             9424.0                0.0                  1.0   
...          ...                ...                ...                  ...   
3035795     10.0             5429.0                0.0                  3.0   
2765287      0.0            12656.0               13.0                 13.0   
2416520    429.0             8293.0                0.0                435.0   
2327407    580.0            29884.0                0.0               5037.0   
2781747  13349.0                0.0               

In [8]:
data.to_csv('data/prep.csv')

# remove un-thresholded features


In [6]:
data = low.copy()

In [7]:
tech = helpers.col_collection(data, 'technique_')
mat = helpers.col_collection(data, 'material_')
loc = helpers.col_collection(data, 'location_')
cols = tech + mat + loc

82 columns found that start with technique_
84 columns found that start with material_
74 columns found that start with location_


In [ ]:
for col in cols:
    if data[col].sum() < 2:
        data.drop(columns=[col], inplace=True)

# resplit test/train


In [10]:
data = prep.copy()

In [11]:
train = data.loc[data['source']=='train'].drop('source',axis=1)
test = data.loc[data['source']=='test'].drop('source',axis=1)

In [12]:
train.to_csv('data/train_prepared.csv')
test.to_csv('data/test_prepared.csv')